# Santander-Winning-Solution# Santander-Winning-Solution

In [1]:
# https://www.kaggle.com/c/santander-customer-transaction-prediction/discussion/89003#latest-520734
import fastai
import time
from fastai.tabular import *
from fastai.text import *
#import feather
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from fastai.callbacks import SaveModelCallback
import logging
from statistics import mean
import datetime

pd.options.mode.chained_assignment = None

In [2]:
data=pickle.load(open("/home/jd/data/santander/train-clean.p", "rb" ) )
#data = pd.read_feather('../input/create-data/921_data.fth')
data = data.set_index('ID_code')

In [3]:
#detd = pd.read_feather('../input/create-data/921_etd.fth')
etd= pickle.load(open("/home/jd/data/santander/test-clean.p", "rb" ) )
etd = etd.set_index('ID_code')

In [5]:
#train_22506

In [6]:
data=data.drop_duplicates()

In [7]:
data[20140:20155]

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_22497,0,9.1579,1.3476,8.5891,6.2748,13.0548,-2.7795,6.2791,17.0315,-1.1331,...,1.3011,9.4621,2.7603,10.6858,18.8804,0.5372,1.8650,8.3785,17.0625,-11.2118
train_22498,0,13.2036,-6.2755,11.4552,8.2712,10.0797,-12.2604,4.2713,14.5865,0.3519,...,3.7233,15.4919,0.6392,5.4669,22.1979,0.7146,4.6797,8.2330,11.6376,-0.5268
train_22499,1,10.5406,-2.7286,12.7717,4.4413,11.2804,2.5777,5.2201,18.1538,-3.9111,...,7.5293,9.1468,0.5688,-3.1201,21.2724,-0.6478,9.3702,8.4423,13.8076,2.9751
train_22500,0,9.7423,-8.0425,9.9778,7.4554,8.8404,12.0145,4.5717,14.3863,2.5103,...,-2.5013,6.3594,0.2185,9.0361,12.8919,-0.6289,7.6022,8.9423,16.5270,-7.8563
train_22501,0,12.4038,-2.0437,7.9909,7.7884,13.5306,0.2073,4.8342,14.7621,0.6379,...,5.6653,5.2894,3.3136,3.2037,18.5870,0.6462,9.3349,9.3588,20.4581,6.5636
train_22502,0,17.3868,-0.4282,7.9560,10.4887,9.1805,-4.9114,4.9846,19.0288,-6.3760,...,2.3812,9.1561,0.8413,11.2369,11.9624,-0.0883,0.3648,10.0720,16.2518,8.4087
train_22503,0,11.7839,-1.9702,12.3756,5.3059,11.4065,1.8952,5.2023,14.6155,-6.3390,...,3.6831,11.0278,3.4165,-1.7981,19.7462,0.1990,1.0309,6.7715,14.4081,-14.2496
train_22504,0,8.3360,1.1623,7.8013,7.7478,13.4251,-1.1874,4.3656,17.1902,4.1468,...,3.3780,4.4131,-0.7277,2.5412,20.1711,-0.3680,-4.3988,8.6304,17.0207,5.8902
train_22505,0,18.0197,-2.6256,7.0583,4.4731,11.8319,-18.0227,6.4057,13.0855,3.5550,...,5.0606,8.1031,1.8951,1.0618,18.9449,0.2953,4.1242,7.9104,13.7115,0.5103


In [8]:
len(data)

200000

### 400 Added features

We created 200 (one per raw feature) categorical features, 
Let's call them "has one feat", with 5 categories that corresponds (for train data) to:

1. This value appears at least another time in data with target==1 and no 0;
2. This value appears at least another time in data with target==0 and no 1;
3. This value appears at least two more time in data with target==0 & 1;
4. This value is unique in data;
5. This value is unique in data + test (only including real test samples);

In [9]:
data_samp=data

In [10]:
data_samp.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200000 entries, train_0 to train_199994
Columns: 201 entries, target to var_199
dtypes: float64(200), int64(1)
memory usage: 308.2+ MB


In [11]:
# for testing lets use this
col = 'var_0'

In [12]:
12.7836 in etd[col].values.tolist() 

False

In [13]:
data_samp[col].tolist()
8.6093

data_samp[col].tolist().count(8.6093)-1

3

In [14]:
data_samp

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
train_5,0,11.4763,-2.3182,12.6080,8.6264,10.9621,3.5609,4.5322,15.2255,3.5855,...,-6.3068,6.6025,5.2912,0.4403,14.9452,1.0314,-3.6241,9.7670,12.5809,-4.7602
train_6,0,11.8091,-0.0832,9.3494,4.2916,11.1355,-8.0198,6.1961,12.0771,-4.3781,...,8.7830,6.4521,3.5325,0.1777,18.3314,0.5845,9.1104,9.1143,10.8869,-3.2097
train_7,0,13.5580,-7.9881,13.8776,7.5985,8.6543,0.8310,5.6890,22.3262,5.0647,...,13.1700,6.5491,3.9906,5.8061,23.1407,-0.3776,4.2178,9.4237,8.6624,3.4806
train_9,0,12.5088,1.9743,8.8960,5.4508,13.6043,-16.2859,6.0637,16.8410,0.1287,...,0.5543,6.3160,1.0371,3.6885,14.8344,0.4467,14.1287,7.9133,16.2375,14.2514


In [15]:
#for row in data_samp.itertuples():
   # print(row[0],row[2])
    #data_samp.loc[row[0], 'var_0'] = 5

In [21]:
def cat(df, col): 
    #Create new column- set to 6l 0 is causing problems? 
    df[col+"_has_one"]=6
    
    #The other 200 (one per raw feature) features are numerical, let's call them "not unique feat", and 
    #correspond to the raw feature replacing values that are unique in data + test with the mean of the feature.
    df[col+"_not_unique"]=0

    
    #New columns name
    has_one=col+"_has_one"
    not_unique=col+"_not_unique"
    
    #Checking where the number is in the column
    newColumn=df.columns.get_loc(has_one)+1
    
    #Hold onto the column name
    cola=col
    
    #list of values for feature
    trainValues=df[col].tolist()
    testValues=etd[col].values
    
    #get mean of all the features in both test and normal combined
    featmean=mean((etd[cola].values.tolist()+df[cola].tolist()))
    
    # col is the index
    col=df.columns.get_loc(col)+1
    
    #for each row
    for row in df.itertuples():
        
        #set defaults to False
        isunique = False
        isintest = False
        
        targetT = False
        targetF = False
        
        
        #gets the value which will be tested (4.222 or 5.2 or -2)
        valueToTest= row[col]
        
        # is this value unique? Count the number of times -1, then you know number of duplicates
        isunique = not(bool(trainValues.count(valueToTest)-1)) 
        
        #is it in test?
        isintest = valueToTest in testValues

        # is in train and test
        if (isunique and isintest):
            df.loc[row[0], has_one] = 5
            
            #Meets Requirement for 2nd feature
            df.loc[row[0], not_unique] = featmean
            continue
        # is in train only
        elif (isunique):
            #stop
            df.loc[row[0], has_one] = 4
            
            #Does not meet requirement for 2nd feature
            df.loc[row[0], not_unique] = valueToTest
            continue
        else:
            
            #Does not meet requirement for 2nd feature
            df.loc[row[0], not_unique] = valueToTest
            
            indexToCheck=df[df[cola]==valueToTest].index.values

            for y in indexToCheck:
                #print ("y is "  +str(y))
                check=df.loc[y].target
                #print (check)
                if check==1:
                    targetT = True
                else:
                    targetF = True
            
            # Apply appropriate case
            if (targetT and targetF):
                #print (str (row[0])+" case 3")
                df.loc[row[0], has_one] = 3
                #catCol[x]=3
            elif targetT:
                #print (str (row[0])+" case 1")
                df.loc[row[0], has_one] = 1
                #catCol[x]=1
            elif targetF:
            #print (str (row[0])+" case 2")
                df.loc[row[0], has_one] = 2
            #catCol[x]=2
            else:
                print ("Somethingweird just happened")
            
    return df

In [22]:
df=data

In [23]:
valueToTest =  11.8091
indexToCheck=df[df[col]==valueToTest].index.values
indexToCheck

array(['train_6', 'train_22506', 'train_112255'], dtype=object)

In [24]:
indexToCheck
for y in indexToCheck:
    print (str (y))
    print (df.loc[y].target)
    if (df.loc[y].target)==1:
        targetT = True
    else:
        targetF = True

train_6
0.0
train_22506
1.0
train_112255
0.0


In [25]:
# Run it on everything

def has_one(df):
    columns=df.columns.tolist()
    columns.remove('target')
    for x in columns:
        print (x)
        print (datetime.datetime.now())
        df=cat(df, x)
    return df

In [26]:
data_samp=has_one(data_samp)

var_0
2019-04-27 13:42:28.849413
var_1
2019-04-27 14:14:49.238756


KeyboardInterrupt: 

In [ ]:
data_samp